<a href="https://colab.research.google.com/github/visa29/learn/blob/main/AutoSense_RF_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Step 1B: Create a synthetic vehicle dataset

np.random.seed(42)  # to make results reproducible

n_samples = 500  # number of sample vehicles

engine_temp = np.random.normal(90, 10, n_samples)          # around 90°C
rpm = np.random.normal(2500, 800, n_samples)               # engine speed
oil_pressure = np.random.normal(3.0, 0.7, n_samples)       # arbitrary units
battery_voltage = np.random.normal(12.5, 0.5, n_samples)   # around 12.5V
vehicle_speed = np.random.normal(60, 25, n_samples)        # km/h
coolant_temp = engine_temp + np.random.normal(0, 5, n_samples)
mileage_km = np.random.normal(60000, 20000, n_samples)     # old-ish cars

# simple rule: the more "bad" conditions, the higher the risk
risk_score = (
    (engine_temp > 100).astype(int) +
    (rpm > 3500).astype(int) +
    (oil_pressure < 2.0).astype(int) +
    (battery_voltage < 12.0).astype(int) +
    (mileage_km > 80000).astype(int)
)

# label: 1 = high failure risk, 0 = low/normal
failure_risk_label = (risk_score >= 2).astype(int)

df = pd.DataFrame({
    "Engine_temp": engine_temp,
    "rpm": rpm,
    "Oil_pressure": oil_pressure,
    "Battery_voltage": battery_voltage,
    "Vehicle_speed": vehicle_speed,
    "Coolant_temp": coolant_temp,
    "Mileage_km": mileage_km,
    "Failure_risk_label": failure_risk_label
})

# show the first 5 rows
df.head()


,Engine_temp,rpm,Oil_pressure,Battery_voltage,Vehicle_speed,Coolant_temp,Mileage_km,Failure_risk_label
0,94.967142,3240.942038,3.979549,12.889181,43.120543,97.820207,21843.848842,0
1,88.617357,4027.533312,3.647244,12.224407,56.387033,93.458280,42792.299784,0
2,96.476885,1381.145941,3.041741,12.090901,40.189502,94.820343,51727.889332,0
3,105.230299,2950.375389,2.547144,12.498313,52.300962,102.169115,97753.753147,1
4,87.658466,1979.485945,3.488756,12.414908,12.659633,82.232714,71131.062491,0


In [ ]:
# Step 2A: Train–test split and Random Forest import

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Features (X) and label (y)
feature_cols = [
    "Engine_temp",
    "rpm",
    "Oil_pressure",
    "Battery_voltage",
    "Vehicle_speed",
    "Coolant_temp",
    "Mileage_km"
]

X = df[feature_cols]
y = df["Failure_risk_label"]

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
# Step 2B: Train the Random Forest model

rf_model = RandomForestClassifier(
    n_estimators=100,   # number of trees
    max_depth=5,        # limit tree depth (prevents overfitting)
    random_state=42
)

rf_model.fit(X_train, y_train)

print("Model training completed!")


Model training completed!


In [ ]:
# Step 3A: Quick evaluation on test set

y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.92      0.99      0.96        86
           1       0.88      0.50      0.64        14

    accuracy                           0.92       100
   macro avg       0.90      0.74      0.80       100
weighted avg       0.92      0.92      0.91       100



In [ ]:
# Step 3B: Function to predict failure risk percentage

import numpy as np

def predict_failure_risk(sensor_input: dict):
    """
    sensor_input: dictionary with keys same as feature_cols.
    Returns: (risk_percentage, risk_label)
    """
    x = np.array([[sensor_input[col] for col in feature_cols]])
    prob_failure = rf_model.predict_proba(x)[0][1]  # probability of class 1
    risk_pct = round(prob_failure * 100, 2)

    if risk_pct >= 70:
        label = "High Risk"
    elif risk_pct >= 40:
        label = "Medium Risk"
    else:
        label = "Low Risk"

    return risk_pct, label


In [ ]:
# Step 3C: Try with a high-risk example vehicle

example_vehicle = {
    "Engine_temp": 105,      # high
    "rpm": 4200,             # high
    "Oil_pressure": 1.8,     # low
    "Battery_voltage": 11.8, # low
    "Vehicle_speed": 80,
    "Coolant_temp": 103,
    "Mileage_km": 90000      # high mileage
}

risk_pct, label = predict_failure_risk(example_vehicle)
print("Failure Risk:", risk_pct, "% |", label)


Failure Risk: 92.49 % | High Risk


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from datetime import datetime, timedelta

In [ ]:
# Diagnosis Agent: uses the ML model
def diagnosis_agent(sensor_input: dict):
    risk_pct, risk_label = predict_failure_risk(sensor_input)
    print(f"Diagnosis Agent: Failure Risk = {risk_pct}% ({risk_label})")
    return risk_pct, risk_label

# Scheduling Agent: decides when to book service based on risk
def scheduling_agent(risk_label: str):
    today = datetime.now()
    if risk_label == "High Risk":
        slot = today + timedelta(days=1)   # urgent: within 1 day
    elif risk_label == "Medium Risk":
        slot = today + timedelta(days=3)   # medium: within 3 days
    else:
        slot = today + timedelta(days=7)   # low: within a week

    slot_str = slot.strftime("%d %b %Y, %I:%M %p")
    print(f"Scheduling Agent: Recommended service slot = {slot_str}")
    return slot_str

# Customer Engagement Agent: prepares what to tell the user
def customer_engagement_agent(risk_pct: float, risk_label: str, slot_str: str):
    if risk_label == "High Risk":
        tone = "urgent"
    elif risk_label == "Medium Risk":
        tone = "important"
    else:
        tone = "advisory"

    message = (
        "Customer Engagement Agent: "
        f"AutoSense has detected a {tone} issue with your vehicle. "
        f"Failure risk is {risk_pct}%. "
        f"We recommend a service on {slot_str}."
    )
    print(message)
    return message

# Feedback Agent: (for now just prints that it logged the event)
def feedback_agent(risk_pct: float, risk_label: str):
    print(
        "Feedback Agent: Logged service recommendation "
        f"with risk {risk_pct}% ({risk_label}) into maintenance history."
    )


In [ ]:
# UEBA Agent: monitors agent activity for anything unusual

ALLOWED_ACTIONS = {
    "Diagnosis Agent": ["read_sensor_data", "run_prediction"],
    "Scheduling Agent": ["propose_slot"],
    "Customer Engagement Agent": ["send_message"],
    "Feedback Agent": ["log_feedback"],
}

def ueba_agent(activity_log):
    """
    activity_log: list of tuples (agent_name, action)
    """
    anomalies = []
    for agent_name, action in activity_log:
        allowed = ALLOWED_ACTIONS.get(agent_name, [])
        if action not in allowed:
            anomalies.append((agent_name, action))

    if anomalies:
        print("UEBA Agent: WARNING – anomalous actions detected!")
        for agent_name, action in anomalies:
            print(f" - {agent_name} performed unexpected action: {action}")
    else:
        print("UEBA Agent: All agent actions normal. No anomalies detected.")


In [ ]:
def master_agent(sensor_input: dict):
    print("Master Agent: Received vehicle data.\n")

    activity_log = []

    # 1. Diagnosis Agent
    activity_log.append(("Diagnosis Agent", "read_sensor_data"))
    activity_log.append(("Diagnosis Agent", "run_prediction"))
    risk_pct, risk_label = diagnosis_agent(sensor_input)
    print()

    # 2. Scheduling Agent
    activity_log.append(("Scheduling Agent", "propose_slot"))
    slot_str = scheduling_agent(risk_label)
    print()

    # 3. Customer Engagement Agent
    activity_log.append(("Customer Engagement Agent","send_message"))
    message = customer_engagement_agent(risk_pct, risk_label, slot_str)
    print()

    # 4. Feedback Agent
    activity_log.append(("Feedback Agent", "log_feedback"))
    feedback_agent(risk_pct, risk_label)
    print()

    # 5. UEBA check
    ueba_agent(activity_log)

    return {
        "risk_pct": risk_pct,
        "risk_label": risk_label,
        "slot": slot_str,
        "message": message
    }


In [ ]:
print("=== AutoSense Agent Workflow Simulation ===\n")
result = master_agent(example_vehicle)

print("\n=== Summary ===")
print("Failure Risk:", result["risk_pct"], "% |", result["risk_label"])
print("Service Slot:", result["slot"])
print("Message to Owner:", result["message"])


=== AutoSense Agent Workflow Simulation ===

Master Agent: Received vehicle data.

Diagnosis Agent: Failure Risk = 92.49% (High Risk)

Scheduling Agent: Recommended service slot = 04 Dec 2025, 12:03 PM

Customer Engagement Agent: AutoSense has detected a urgent issue with your vehicle. Failure risk is 92.49%. We recommend a service on 04 Dec 2025, 12:03 PM.

Feedback Agent: Logged service recommendation with risk 92.49% (High Risk) into maintenance history.

UEBA Agent: All agent actions normal. No anomalies detected.

=== Summary ===
Failure Risk: 92.49 % | High Risk
Service Slot: 04 Dec 2025, 12:03 PM
Message to Owner: Customer Engagement Agent: AutoSense has detected a urgent issue with your vehicle. Failure risk is 92.49%. We recommend a service on 04 Dec 2025, 12:03 PM.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# A low-risk example vehicle
example_vehicle_low = {
    "Engine_temp": 85,        # normal
    "rpm": 2000,              # normal
    "Oil_pressure": 3.5,      # good
    "Battery_voltage": 12.7,  # good
    "Vehicle_speed": 50,      # normal
    "Coolant_temp": 82,       # normal
    "Mileage_km": 40000       # mid-level mileage
}

print("=== AutoSense Agent Workflow Simulation (LOW RISK) ===\n")
result_low = master_agent(example_vehicle_low)

print("\n=== Summary (Low Risk Vehicle) ===")
print("Failure Risk:", result_low["risk_pct"], "% |", result_low["risk_label"])
print("Service Slot:", result_low["slot"])
print("Message to Owner:", result_low["message"])


=== AutoSense Agent Workflow Simulation (LOW RISK) ===

Master Agent: Received vehicle data.

Diagnosis Agent: Failure Risk = 0.28% (Low Risk)

Scheduling Agent: Recommended service slot = 10 Dec 2025, 12:06 PM

Customer Engagement Agent: AutoSense has detected a advisory issue with your vehicle. Failure risk is 0.28%. We recommend a service on 10 Dec 2025, 12:06 PM.

Feedback Agent: Logged service recommendation with risk 0.28% (Low Risk) into maintenance history.

UEBA Agent: All agent actions normal. No anomalies detected.

=== Summary (Low Risk Vehicle) ===
Failure Risk: 0.28 % | Low Risk
Service Slot: 10 Dec 2025, 12:06 PM
Message to Owner: Customer Engagement Agent: AutoSense has detected a advisory issue with your vehicle. Failure risk is 0.28%. We recommend a service on 10 Dec 2025, 12:06 PM.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [25]:
# Step 3: Simple text-based "dashboard" view for screenshots / demo

def dashboard_view(sensor_input: dict, title="AutoSense Vehicle Health Dashboard"):
    print("=" * 60)
    print(title)
    print("=" * 60)

    # 1. Show sensor inputs
    print("\n[Sensor Inputs]")
    for key, value in sensor_input.items():
        print(f"  - {key}: {value}")

    # 2. Run prediction + schedule + message, but capture results
    #    (we reuse our earlier agent functions, but keep this view clean)

    # Diagnosis
    risk_pct, risk_label = predict_failure_risk(sensor_input)

    # Scheduling
    slot_str = scheduling_agent(risk_label)

    # Customer Message (we call the function but don't want agent prefix twice)
    if risk_label == "High Risk":
        tone = "urgent"
    elif risk_label == "Medium Risk":
        tone = "important"
    else:
        tone = "advisory"

    voice_message = (
        f"AutoSense has detected a {tone} issue with your vehicle. "
        f"Failure risk is {risk_pct}%. "
        f"We recommend a service on {slot_str}."
    )

    # UEBA status – here we just say "Normal", but you can change later
    ueba_status = "All agent actions normal. No anomalies detected."

    # 3. Show prediction summary
    print("\n[Prediction]")
    print(f"  - Failure Risk: {risk_pct}%")
    print(f"  - Risk Level  : {risk_label}")

    # 4. Show recommended service slot
    print("\n[Recommended Service]")
    print(f"  - Suggested Slot: {slot_str}")

    # 5. Show voice agent message
    print("\n[Voice Agent Message]")
    print("  ", voice_message)

    # 6. Show UEBA status
    print("\n[UEBA Status]")
    print("  ", ueba_status)

    print("=" * 60)

    # also return results in case we need them
    return {
        "risk_pct": risk_pct,
        "risk_label": risk_label,
        "slot": slot_str,
        "voice_message": voice_message,
        "ueba_status": ueba_status
    }


In [26]:
# Dashboard for the high-risk vehicle
dashboard_high = dashboard_view(example_vehicle, title="AutoSense Dashboard – High-Risk Vehicle")


AutoSense Dashboard – High-Risk Vehicle

[Sensor Inputs]
  - Engine_temp: 105
  - rpm: 4200
  - Oil_pressure: 1.8
  - Battery_voltage: 11.8
  - Vehicle_speed: 80
  - Coolant_temp: 103
  - Mileage_km: 90000
Scheduling Agent: Recommended service slot = 04 Dec 2025, 12:09 PM

[Prediction]
  - Failure Risk: 92.49%
  - Risk Level  : High Risk

[Recommended Service]
  - Suggested Slot: 04 Dec 2025, 12:09 PM

[Voice Agent Message]
   AutoSense has detected a urgent issue with your vehicle. Failure risk is 92.49%. We recommend a service on 04 Dec 2025, 12:09 PM.

[UEBA Status]
   All agent actions normal. No anomalies detected.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [27]:
# Dashboard for the low-risk vehicle
dashboard_low = dashboard_view(example_vehicle_low, title="AutoSense Dashboard – Low-Risk Vehicle")


AutoSense Dashboard – Low-Risk Vehicle

[Sensor Inputs]
  - Engine_temp: 85
  - rpm: 2000
  - Oil_pressure: 3.5
  - Battery_voltage: 12.7
  - Vehicle_speed: 50
  - Coolant_temp: 82
  - Mileage_km: 40000
Scheduling Agent: Recommended service slot = 10 Dec 2025, 12:10 PM

[Prediction]
  - Failure Risk: 0.28%
  - Risk Level  : Low Risk

[Recommended Service]
  - Suggested Slot: 10 Dec 2025, 12:10 PM

[Voice Agent Message]
   AutoSense has detected a advisory issue with your vehicle. Failure risk is 0.28%. We recommend a service on 10 Dec 2025, 12:10 PM.

[UEBA Status]
   All agent actions normal. No anomalies detected.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
